# Setting the fundarmental group of $\Sigma_{g,1}$ 

In [1]:
require './freegroup.rb'

true

## generators

In [2]:
genus = 3
alphabet = %w(a b c d e f)

Gens = [Letter.new] + alphabet.map{|x| Letter.new(x)}
Gens.freeze
p Gens[1..-1].map(&:show)

a = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:first)
b = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:last)
p a.map(&:show)
nil

["a", "b", "c", "d", "e", "f"]
["1", "a", "c", "e"]


In [3]:
comms = []
(Gens.size/2 + 1).times do |i|
  comms << Group.commutator(a[i], b[i])
end
p comms.map(&:show)

["1111", "abAB", "cdCD", "efEF"]


["1111", "abAB", "cdCD", "efEF"]

In [4]:
Rseq = []
(Gens.size/2).times do |i|
  Rseq += [a[i+1], b[i+1].inverse, a[i+1].inverse, b[i+1]]
end
Rseq.freeze
p Rseq.map(&:show) #.index('A')
nil

<main>:2: warning: already initialized constant Rseq
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here


["a", "B", "A", "b", "c", "D", "C", "d", "e", "F", "E", "f"]


# The Algorithm

In [383]:
#myw = Word.new(a[1], b[2]) #.inverse)
#myw = Word.new(a[1])^2
c = Word.new(a[1], b[1], a[1].inverse); myw = c
#myw = comms[1]
myw = Word.new(a[1])*(comms[2]^2)
#myw = Word.new(a[1])*comms[2]*c*comms[2]*(c.inverse)
p myw.show

"(a)((cdCD)(cdCD))"


"(a)((cdCD)(cdCD))"

In [384]:
box = {}
Rseq.each{|x| box.store(x.char,[])}

facs = myw.flatten.factors

facs.each_with_index do |c, i|
  k = i
  box[c.char].insert(-1, 2*k)
  box[c.inverse.char].insert(0,2*k+1)
end
p box, box.values.map(&:to_s).join
nil

{"a"=>[0], "B"=>[], "A"=>[1], "b"=>[], "c"=>[15, 7, 2, 10], "D"=>[13, 5, 8, 16], "C"=>[11, 3, 6, 14], "d"=>[17, 9, 4, 12], "e"=>[], "F"=>[], "E"=>[], "f"=>[]}
"[0][][1][][15, 7, 2, 10][13, 5, 8, 16][11, 3, 6, 14][17, 9, 4, 12][][][][]"


In [400]:
p myw.show
p box.values.map(&:to_s).join

bvf = box.values.flatten
size = bvf.size

odd_pairs=(1..size/2).to_a.map{|x| 2*x-1}.combination(2)

coprods = odd_pairs.map do |pair|
  pair_of_succs = [[pair[0], pair[0]+1], [pair[1], (pair[1]+1)%size]]
  order = bvf.select{|n| pair_of_succs.flatten.include?(n)}
  
  # give the parity of order
  idxs = pair_of_succs.map{|p| p.map{|n| order.index(n)}}
  p "#{pair_of_succs.to_s}, #{order.to_s}, #{idxs.to_s}" if order == [10, 3, 9, 4]
  rng = idx[0].sort
  parity = order.slice(rng[0]..rng[1]).size.even? ? 'even' : 'odd'
  
  # give the sign of order
  sign0 = (idxs[0][0] < idxs[0][1])
  sign1 = (idxs[1][0] < idxs[1][1])
  sign2 = (idxs[0][0] < idxs[1][0])
  sign = [sign0, sign1, sign2]
  
  fac1 = box.map{|key, arr| [key, arr.select{|x| x <= pair[0] or pair[1] < x}]}.to_h
  fac2 = box.map{|key, arr| [key, arr.select{|x| pair[0] < x and x <= pair[1]}]}.to_h
  # contract fac1 & fac2
  #
  # shift numbers in fac1 & fac2
  factors = [fac1, fac2].map do |f|
    sorted = f.values.flatten.sort
    f.map{|key, arr| [key, arr.map{|v| sorted.index(v)}]}.to_h
  end
  # code --> word
  words = [fac1, fac2].map do |f|
    chars = (0..size/2).to_a.map{|i| f.map{|key, arr| key if arr.include?(2*i)}.join}
    Word.new(chars.join)
  end
  {pair: pair, order: order, parity: parity, sign: sign, factors: factors, words: words}
end
  
printf("----------------------\n")
#  p coprods

coprods.each do|h|
  if h[:parity] == 'odd' then
    strings = h[:factors].map{|h| h.values.map(&:to_s).join}
    #----------
#    p "#{h[:pair]}"
    p "#{h[:order]}"
#    p "#{h[:parity]}"
    p "#{h[:sign]}"
#    p strings.join(" (x) ")
#    p "#{h[:words][0]} (x) #{h[:words][1]}"
    p "#{h[:words][0].contract} (x) #{h[:words][1].contract}"
    printf("---------------\n")
  else
#    p nil
  end
end
nil

"(a)((cdCD)(cdCD))"
"[0][][1][][15, 7, 2, 10][13, 5, 8, 16][11, 3, 6, 14][17, 9, 4, 12][][][][]"
"[[3, 4], [9, 10]], [10, 3, 9, 4], [[1, 3], [2, 0]]"
----------------------
"[1, 7, 2, 8]"
"[true, true, true]"
"aDcdCD (x) cdC"
---------------
"[1, 15, 2, 16]"
"[true, true, true]"
"aD (x) cdCDcdC"
---------------
"[5, 3, 6, 4]"
"[true, true, false]"
"aDcdCD (x) d"
---------------
"[10, 3, 9, 4]"
"[true, false, true]"
"accdCD (x) dCD"
---------------
"[13, 3, 14, 4]"
"[true, true, false]"
"aD (x) dCDcd"
---------------
"[0, 3, 17, 4]"
"[true, false, true]"
"ac (x) dCDcdCD"
---------------
"[7, 5, 8, 6]"
"[true, true, false]"
"accdCD (x) C"
---------------
"[5, 11, 6, 12]"
"[true, true, true]"
"acddCD (x) CDc"
---------------
"[15, 5, 16, 6]"
"[true, true, false]"
"ac (x) CDcdC"
---------------
"[7, 10, 8, 9]"
"[true, false, true]"
"acddCD (x) D"
---------------
"[7, 13, 8, 14]"
"[true, true, true]"
"acdCCD (x) Dcd"
---------------
"[13, 11, 14, 12]"
"[true, true, false]"
"acdCDD (x) d"
--

In [332]:
arr =[5, 8, 6, 7].combination(2).map do |com|
  p com
  (com[0] > com[1]) ? 1 : 0
end
p arr
arr.sum.odd?

[5, 8]
[5, 6]
[5, 7]
[8, 6]
[8, 7]
[6, 7]
[0, 0, 0, 1, 1, 0]


false